# Estimate Pi by tossing needles and Python

This post estimates the constant Pi by "tossing" needles at a board with equal sized strips seperated by the same width (Figure below). This is called "Buffon's needle problem" and it was posed by <a href="https://en.wikipedia.org/wiki/Buffon%27s_needle_problem"> Georges-Louis Leclerc, Comte de Buffon</a>:

<i>Suppose we have a floor made of parallel strips of wood, each the same width, and we drop a needle onto the floor. What is the probability that the needle will lie across a line between two strips?</i>

<a href="https://mathworld.wolfram.com/BuffonsNeedleProblem.html"> Several proofs</a> have been shown the probaility is $\frac{2d}{\Pi l}$. So, if we drop n needles and there are c crosses, we can re-arrange that formula to be:

$\Pi = \frac{2*d*n}{l*c}$

In [1]:
## OpenCV (Open Source Computer Vision Library) to visualize the simulation
import cv2 as cv2 

## Numpy for quick mathematical operations and random number generation
import numpy as np 
from numpy import random

## Math for some trigonometry operations
import math

import time

In [2]:
# This list will be appended each time a needle crosses a line
crossedList=[]

#This lists will contain all the points plotted on the graph "img2"
points=[]

# The number of pixels seperating each line
distance=90

# Number of lines plotted 
nLines=6

# The pixel y-value where each of the six lines will be ploted 
lines=[]
for i in range(0,nLines):
    lines.append((distance)*i+90)

# This is a counter variable that will be updated each time an element is popped from the "points" list.
# This will allow us to shift the graph to the right.
N_popped=0

# Counter variables for the number of throws and value of the x-axis to be plotted on img2. Note, the first
# plotted value will be at 50 throws.
throws=0
throws_x=50

# This maps the values between 2.5 and 3.5 (used for y-axis in img2) to pixel values betweeen 0 and 600.
# Because pixel 0,0 is the top left corner, we have to reverse the mapping so the value 3.50 is on the 
# top and 2.5 is on the bottom of the y-axis.
mappedList=np.around(np.linspace(3.5,2.5,600,dtype=float), decimals=3)

# The size of the window is the (number of lines + 1) multiplied by the distance between lines.
# The windows will be 630 pixels x 630 pixels.
windowLength=distance*(nLines+1)

# # We initiate image 1 as a black screen
img = np.zeros((windowLength,windowLength,3), np.uint8)

# We set the font 
font = cv2.FONT_HERSHEY_SIMPLEX

# Create tick marks/labels for the y-axis graph in img2. 
ticks=[2.5, 2.6, 2.7, 2.8, 2.9, 3,3.1,3.141,3.2,3.3,3.4,3.5]
labels=['2.50', '2.60', '2.70', '2.80', '2.90', '3.00','3.10','3.141','3.20','3.30','3.40','3.50']

# Initiate and keep the the simualtion running until the letter "q" is pressed.
start=time.process_time ()
while True:
    #Increase the number of throws by 1
    throws+=1
    
    # Randomly choose an angle between 0 and 360 degress and convert it to radians.
    theta=math.radians(random.uniform(0,360))
    
    # Randomly choose an x value for a point between 0 and 630 pixels. 
    # Note: This is the width of the window
    ptx1=random.uniform(min(lines)-90,max(lines)+90)
    
    # Randomly choose an y value for a point between 90 and 540 pixels.
    # Note: This is the height between the top and bottom horizontal lines in "img"
    pty1=random.uniform(min(lines),max(lines))
    
    # Our aim is to create a line that is 45 pixels long. 
    # We can use trigonometry to identify the second point (see Figure 4).
    # A striaght line with an angle theta (from above) and 
    ptx2=(ptx1+math.cos(theta)*(distance/2))
    pty2=(pty1+math.sin(theta)*(distance/2))
    
    # We have to convert all points to integers because OpenCV is unable to plot float values 
    x1,y1,x2,y2=(int(round(ptx1,0)),int(round(pty1,0)),int(round(ptx2,0)),int(round(pty2,0)))
    
    # If the needle crosses a line if the y1 and y2 values are above and below any of the six
    # horizontal lines (see Figure 1). A value of "True" is appended to the crossedList if the 
    # needle crosses a line. Otherwise, "False" is appended to the list. 
    found = list(filter(lambda x: (pty1 <= x <= pty2) or (pty2 <= x <= pty1), lines))
    crossedList.append(len(found)>0)
    
    # We update the estimate for the value of Pi after each throw  
    Pi_est=throws/(sum(crossedList)+0.00001)
    Pi_est_y=round(Pi_est,3)

    # The value of Pi will vary a lot for the first few throws. As such, we will bound it between 
    # 2.5 and 3.5 for the purpose of plotting on "img2.
    # The "y_value" is the index where the estimate of Pi is equal to an value in the "mappedList"
    # For example, if Pi_est=3.500, then y_value = 0 (top pixel on the screen). If Pi_est=2.500, then y_value=600. 
    if (Pi_est_y>=2.50) and (Pi_est_y<=3.5):
        y_value=np.where(mappedList >= Pi_est_y)[0][-1]
    elif Pi_est_y<2.50:
        y_value=int(0)
    elif Pi_est_y>3.5:
        y_value=int(600)
    
    #Randomly choose a blue, green, and red color for the needle in "img".
    colorBlu=random.randint(0,255)
    colorGrn=random.randint(0,255)
    colorRed=random.randint(0,255)
    
    # Display the number of throws and estimate of Pi in "img".
    cv2.putText(img, 'Est Pi: '+ str(Pi_est), (int(windowLength*0.779),20), font, 0.6, (0, 255, 0), 2)
    cv2.putText(img, "n = "+str(throws), (10,20), font, 0.5, (0, 255, 0), 2) ### (B,G,R)
    
     
    # draw the needle in "img".
    cv2.line(img,(x1,y1),(x2,y2),(colorBlu,colorGrn,colorRed),2)
    for line in lines:
        # Draw a horizontal blue line with thickness of 2 px
        cv2.line(img,(0,line),(windowLength,line),(255,255,255),2)
    
#     # Show img and label the window
#     cv2.imshow('Estimate Pi using Buffon Needle Simulation', img)
    
    # Draw black background rectangle on top text from previous throws. 
    cv2.rectangle(img, (int(windowLength*0.75), 0), (int(windowLength), 60), (0, 0, 0), -1)
    cv2.rectangle(img, (0, 0), (100, 40), (0, 0, 0), -1)
    
    # Create/update img2 every 50 throws.
    if (throws % 50 == 0):
        throws_x+=1
        points.append((throws_x, y_value))
        img2 = np.zeros((windowLength,windowLength,3), np.uint8)
        
        # Create an x- and y-axes
        cv2.line(img2,(50,600),(windowLength,600),(255,255,255),2)
        cv2.line(img2,(50,0),(50,600),(255,255,255),2)
        # draw a horizontal line at the value 3.141
        cv2.line(img2,(50,np.where(mappedList >= 3.141)[0][-1]),(windowLength,np.where(mappedList >= 3.141)[0][-1]),(255,255,255),1)
        
        # Draw tick marks
        for tick,label in zip(ticks,labels):
            y_tick=np.where(mappedList >= tick)[0][-1]
            cv2.line(img2,(47,y_tick),(53,y_tick),(255,255,255),1)
            cv2.putText(img2, label, (0,y_tick+3), font, 0.5, (255, 255, 255), 1)

        # if there are more than 600 points on the graph (i.e., the length of "points" is 600,
        # then remove the first element in the list and increase the "N_popped" counter by 1.
        # The "N_popped" counter will allow us to shift the graph to the right so the plotted points
        # do not go off the window/screen.
        if throws_x>600:
            points.pop(0)
            N_popped+=1
        
        # Update img2 by putting the respective texts and plotting a circle based on the 
        # current estimate of Pi.
        for point in points:
            cv2.rectangle(img2, (int(windowLength*0.75), 0), (int(windowLength), 60), (0, 0, 0), -1)
            cv2.rectangle(img2, (0, 0), (100, 40), (0, 0, 0), -1)
            cv2.putText(img2, "n = "+str(throws), (10,20), font, 0.5, (0, 255, 0), 2) ### (B,G,R)
            cv2.putText(img2, 'Est Pi: '+ str(Pi_est), (int(windowLength*0.779),20), font, 0.6, (0, 255, 0), 2)
            cv2.circle(img2, (point[0]-(N_popped),point[1]), 1, (0, 255, 0), -1)
            cv2.imshow('Graph', img2)

    # Continue updating/running the similulation until the letter "q" is pressed
    # or until 100,000 throws.
    if cv2.waitKey(1) == ord('q'):
        cv2.destroyAllWindows()
        break
        
    if throws==100000:
        cv2.destroyAllWindows()
        break
    
    # If you want, you can run the simulation slower by uncommenting the sleep function below.
#     time.sleep(1) 

# Print the number of throws and estimate of Pi upon exit from the simulation.
print(f'Number of needles: {throws}')
print(f'Number of crosses: {sum(crossedList)}')
print(f'Estimated Pi: {throws/sum(crossedList)}')
print('Time in seconds:',time.process_time()-start)

Number of needles: 10033
Number of crosses: 3231
Estimated Pi: 3.1052305787681833
Time in seconds: 81.84375


The code below runs 314 simulations of 3141 tosses and estimates Pi at 3.1409. The simulations are not visualized.

In [3]:
nSimulations=314
nThrows=3141

Simulations=[]

# The number of pixels seperating each line
distance=90

# Number of lines plotted 
nLines=6

# The pixel y-value where each of the six lines will be ploted 
lines=[]
for i in range(0,nLines):
    lines.append((distance)*i+90)
    
random.seed(143202022)

for j in range(0,nSimulations):
    crossedList=[]
    for i in range(0,nThrows):
        angle=math.radians(random.uniform(0,360))
        ptx1=random.uniform(min(lines)-90,max(lines)+90)
        pty1=random.uniform(min(lines),max(lines))
        ptx2=(ptx1+math.cos(angle)*(distance/2))
        pty2=(pty1+math.sin(angle)*(distance/2))
        x1,y1,x2,y2=(int(round(ptx1,0)),int(round(pty1,0)),int(round(ptx2,0)),int(round(pty2,0)))
        found = list(filter(lambda x: (pty1 <= x <= pty2) or (pty2 <= x <= pty1), lines))
        crossedList.append(len(found)>0)
    Pi_est=i/(sum(crossedList))
    Simulations.append(Pi_est)

print(f'Number of simulations: {nSimulations}')
print(f'Number of throws per simulation: {nThrows}')
print(f'Estimated Pi: {np.mean(Simulations)}')

Number of simulations: 314
Number of throws per simulation: 3141
Estimated Pi: 3.1409124701339937
